In [1]:
import numpy as np
from stable_baselines3 import PPO
# from sb3_contrib import TRPO
from gymnasium import spaces, Env
from boat_simulation import Boat, wrap_phase

2024-01-18 02:48:23.315466: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-18 02:48:23.334319: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 02:48:23.334337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-18 02:48:23.334872: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-18 02:48:23.338183: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import torch
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor


def linear(device, x, in_units, out_units, activation):
    x = nn.Linear(in_units, out_units).to(device)(x)
    return activation().to(device)(x)

def mlp(device, x, units):
    for i in range(0, len(units)):
        if not isinstance(units[i], tuple):
            units[i] = (units[i], nn.ReLU)

    for i in range(1, len(units)):
        prev_layer_size, _ = units[i - 1]
        layer_size, activation = units[i]
        x = linear(device, x, prev_layer_size, layer_size, activation)
    return x

class CustomMLP(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=1):
        super(CustomMLP, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def forward(self, observations):
        x = observations.to(self.device)
        return mlp(self.device, x, [7, 400, 300, (self.features_dim, nn.Tanh)])


# Define the LSTM model
class NonLSTM(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim):
        super(NonLSTM, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.hidden_layer_size = 64
        x1, x2 = observation_space.shape
        input_size = x1 * x2
        self.linear = lambda x: mlp(self.device, x, [input_size, self.hidden_layer_size, self.hidden_layer_size, features_dim])

    def forward(self, input_seq):
        batch_size = input_seq.shape[0]
        input = input_seq.view(batch_size, -1)
        predictions = self.linear(input)
        return predictions

# Define the LSTM model
class LSTMModel(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim):
        super(LSTMModel, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.hidden_layer_size = 64
        (seq_size, input_size,) = observation_space.shape
        self.lstm = nn.LSTM(input_size, self.hidden_layer_size, num_layers=3, batch_first=True).to(self.device)

        self.linear = lambda x: mlp(self.device, x, [self.hidden_layer_size, self.hidden_layer_size, features_dim])

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out[:, -1, :].view(-1, self.hidden_layer_size))
        return predictions

In [3]:
from environments import MultiMarkEnv

In [4]:
from IPython.core.display import display, HTML, Javascript

disable_scroll_script = """
var idx = Jupyter.notebook.get_selected_index();
var cell = Jupyter.notebook.get_cell(idx);
cell.output_area._should_scroll = function(lines) {
    return false;
}
"""

display(Javascript(disable_scroll_script))

/tmp/ipykernel_230485/738611061.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, Javascript


<IPython.core.display.Javascript object>

In [5]:
import plotly.graph_objects as go
import time
import ipywidgets as widgets
from IPython.display import display

outer_radius = 2 * 250 + 2.5 * 0.1 * 250

dt = 15
num_marks = 2
max_seconds = 500 * num_marks / dt

# Initialize the figure and scatter plot
fig = go.FigureWidget()
scatter = fig.add_scatter(mode='markers+lines', name='trajectory')
marks_scatter = fig.add_scatter(mode='markers+text', name='marks')
fig.update_xaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.update_yaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.layout.width=800
fig.layout.height=800

heading_fig = go.FigureWidget()
heading_scatter = heading_fig.add_scatter(mode='markers+lines', name='heading')
rudder_angle_scatter = heading_fig.add_scatter(mode='markers+lines', name='rudder angle')
heading_fig.update_xaxes(range=[0, max_seconds],dtick=25)
heading_fig.update_yaxes(range=[-210, 210],dtick=30)
heading_fig.layout.width=800
heading_fig.layout.height=600
heading_fig.layout.title = "Heading over time"

vmg_fig = go.FigureWidget()
vmg_scatter = vmg_fig.add_scatter(mode='markers+lines')
vmg_fig.update_xaxes(range=[0, max_seconds],dtick=25)
vmg_fig.update_yaxes(range=[-4, 4],dtick=2)
vmg_fig.layout.width=800
vmg_fig.layout.height=400
vmg_fig.layout.title = "VMG over time"

# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

episode = 0

colormap = ['blue', 'red', 'green', 'orange', 'black']

def plot(data, marks, bounds):
    global episode, out, fig
    episode += 1
    if episode % 50 != 0:
        return
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds


    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    rudder_angles = [point['meta']['rudder_angle'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    rudder_angle_scatter = heading_fig.data[1]
    rudder_angle_scatter.x = list(range(len(rudder_angles)))
    rudder_angle_scatter.y = rudder_angles
    rudder_angle_scatter.marker.color = 'lightgreen'
    rudder_angle_scatter.line.color = 'lightgreen'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")

None

Output()

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'trajectory',
              'type': 'scatter',
              'uid': '08f6d830-76c2-418b-9758-db82de8f0298'},
             {'mode': 'markers+text', 'name': 'marks', 'type': 'scatter', 'uid': 'a1eabe1e-ab7f-4af4-bf6c-6586650b5fe5'}],
    'layout': {'height': 800,
               'template': '...',
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [-562.5, 562.5]},
               'yaxis': {'dtick': 25, 'range': [-562.5, 562.5]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'heading',
              'type': 'scatter',
              'uid': '96268745-33f5-44bd-9115-ef82767bfd14'},
             {'mode': 'markers+lines',
              'name': 'rudder angle',
              'type': 'scatter',
              'uid': 'e674a4dd-f3d5-446d-bbb8-3f119f1ffdee'}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'Heading over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 66.66666666666667]},
               'yaxis': {'dtick': 30, 'range': [-210, 210]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines', 'type': 'scatter', 'uid': '6650dde8-a86c-4a9b-89c4-158e34423a86'}],
    'layout': {'height': 400,
               'template': '...',
               'title': {'text': 'VMG over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 66.66666666666667]},
               'yaxis': {'dtick': 2, 'range': [-4, 4]}}
})

In [6]:
# Configuration

r = 250
actions = np.array([-90, -45, -15, 0, 15, 45, 90]) / 180.0

config = {
    'max_marks': 1,
    'max_seconds_per_leg': 500,
    'plot_fn': plot,
    'leg_radius': r,
    'actions': actions,
    'target_tolerance_multiplier': 5
}

outer_radius = config['max_marks'] * r + 0.1 * r
bounds = [-outer_radius, outer_radius, -outer_radius, outer_radius]
# Initialize Environment
env = MultiMarkEnv(config, dt=15, bounds=bounds)

# Initialize PPO model
model = PPO("MlpPolicy", env, verbose=1, device='cuda', policy_kwargs={
    "net_arch": [64, 400, 300]
})

# Train the model
# model.learn(total_timesteps=1_000_000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
model.learn(total_timesteps=1_500_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.2     |
|    ep_rew_mean     | -0.684   |
| time/              |          |
|    fps             | 1082     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 18.6        |
|    ep_rew_mean          | -0.584      |
| time/                   |             |
|    fps                  | 988         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010712819 |
|    clip_fraction        | 0.12        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.94       |
|    explained_variance   | -0.0514     |
|    learning_rate        | 0.

In [8]:
model.save("MultiMarkPPO_boat_model_coarse.dat")

In [9]:
import plotly.graph_objects as go
import time
import ipywidgets as widgets
from IPython.display import display

outer_radius = 2 * 250 + 2.5 * 0.1 * 250

dt = 5
num_marks = 2
max_seconds = 500 * num_marks / dt

# Initialize the figure and scatter plot
fig = go.FigureWidget()
scatter = fig.add_scatter(mode='markers+lines', name='trajectory')
marks_scatter = fig.add_scatter(mode='markers+text', name='marks')
fig.update_xaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.update_yaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.layout.width=800
fig.layout.height=800

heading_fig = go.FigureWidget()
heading_scatter = heading_fig.add_scatter(mode='markers+lines', name='heading')
rudder_angle_scatter = heading_fig.add_scatter(mode='markers+lines', name='rudder angle')
heading_fig.update_xaxes(range=[0, max_seconds],dtick=25)
heading_fig.update_yaxes(range=[-210, 210],dtick=30)
heading_fig.layout.width=800
heading_fig.layout.height=600
heading_fig.layout.title = "Heading over time"

vmg_fig = go.FigureWidget()
vmg_scatter = vmg_fig.add_scatter(mode='markers+lines')
vmg_fig.update_xaxes(range=[0, max_seconds],dtick=25)
vmg_fig.update_yaxes(range=[-4, 4],dtick=2)
vmg_fig.layout.width=800
vmg_fig.layout.height=400
vmg_fig.layout.title = "VMG over time"

# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

episode = 0

colormap = ['blue', 'red', 'green', 'orange', 'black']

def plot_new(data, marks, bounds):
    global episode, out, fig
    episode += 1
    if episode % 50 != 0:
        return
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds


    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    rudder_angles = [point['meta']['rudder_angle'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    rudder_angle_scatter = heading_fig.data[1]
    rudder_angle_scatter.x = list(range(len(rudder_angles)))
    rudder_angle_scatter.y = rudder_angles
    rudder_angle_scatter.marker.color = 'lightgreen'
    rudder_angle_scatter.line.color = 'lightgreen'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")

None

Output()

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'trajectory',
              'type': 'scatter',
              'uid': '5746a639-364f-491e-9bc5-a85496c3fa59'},
             {'mode': 'markers+text', 'name': 'marks', 'type': 'scatter', 'uid': 'c8fd1459-1aa9-4812-b859-0186afaca6b9'}],
    'layout': {'height': 800,
               'template': '...',
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [-562.5, 562.5]},
               'yaxis': {'dtick': 25, 'range': [-562.5, 562.5]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'heading',
              'type': 'scatter',
              'uid': 'ee15fb37-6b01-4f9c-9000-26b4135a6063'},
             {'mode': 'markers+lines',
              'name': 'rudder angle',
              'type': 'scatter',
              'uid': '0abd6928-ff83-4775-8b5c-4c717d416f7d'}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'Heading over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 200.0]},
               'yaxis': {'dtick': 30, 'range': [-210, 210]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines', 'type': 'scatter', 'uid': '6bb73dad-61ba-4731-a50e-1c74bfb3b83d'}],
    'layout': {'height': 400,
               'template': '...',
               'title': {'text': 'VMG over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 200.0]},
               'yaxis': {'dtick': 2, 'range': [-4, 4]}}
})

In [10]:
config_new = {**config}
config_new['plot_fn'] = plot_new
config_new['target_tolerance_multiplier'] = 2

model.set_env(MultiMarkEnv(config, dt=5, bounds=bounds))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
model.learn(total_timesteps=1_500_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 49.4     |
|    ep_rew_mean     | 3.74     |
| time/              |          |
|    fps             | 1833     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 54.1        |
|    ep_rew_mean          | 3.62        |
| time/                   |             |
|    fps                  | 1518        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.056554474 |
|    clip_fraction        | 0.293       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.624      |
|    explained_variance   | 0.404       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.000729   |
|    n_updates            | 7340        |
|    policy_gradient_loss | -0.0294     |
|    value_loss           | 0.128       |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 54    

In [12]:
model.save("MultiMarkPPO_boat_model_fine.dat")

In [47]:
env = MultiMarkEnv(config, dt=5, bounds=bounds)


def render(data, marks, bounds):
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds


    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    rudder_angles = [point['meta']['rudder_angle'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    rudder_angle_scatter = heading_fig.data[1]
    rudder_angle_scatter.x = list(range(len(rudder_angles)))
    rudder_angle_scatter.y = rudder_angles
    rudder_angle_scatter.marker.color = 'lightgreen'
    rudder_angle_scatter.line.color = 'lightgreen'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")


# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

Output()

FigureWidget({
    'data': [{'hovertext': [{'current_mark': '0.000', 'vmg': '0.000', 'heading':
                            '152.163', 'rudder_angle': '15.000', 'reward':
                            '0.000'}, {'current_mark': '0.000', 'vmg': '-2.715',
                            'heading': '154.892', 'rudder_angle': '15.000',
                            'reward': '-0.272'}, {'current_mark': '0.000', 'vmg':
                            '-2.346', 'heading': '133.008', 'rudder_angle':
                            '-90.000', 'reward': '-0.235'}, {'current_mark':
                            '0.000', 'vmg': '-2.783', 'heading': '155.357',
                            'rudder_angle': '90.000', 'reward': '-0.278'},
                            {'current_mark': '0.000', 'vmg': '-2.484', 'heading':
                            '133.523', 'rudder_angle': '-90.000', 'reward':
                            '-0.248'}, {'current_mark': '0.000', 'vmg': '-2.816',
                            'heading': '155.83

FigureWidget({
    'data': [{'line': {'color': 'lightgrey'},
              'marker': {'color': [blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, red]},
              'mode': 'markers+lines',
              'name': 'heading',
              'type': 'scatter',
              'uid': 'ee15fb37-6b01-4f9c-9000-26b4135a6063',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36,

FigureWidget({
    'data': [{'line': {'color': 'lightgrey'},
              'marker': {'color': [blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, red]},
              'mode': 'markers+lines',
              'type': 'scatter',
              'uid': '6bb73dad-61ba-4731-a50e-1c74bfb3b83d',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 

In [50]:
target_x = 0
target_y = 250

best_trajectory = None
best_reward = None
for i in range(20):
    obs, _ = env.reset()
    env.target_x[0] = target_x
    env.target_y[0] = target_y
    env.boat.heading = np.pi / 4
    env.heading = np.pi/4
    done = False
    total_reward = 0
    while not done:
        action, _states = model.predict(obs)
        obs, reward, is_terminal, is_truncated, info = env.step(action)
        done = is_terminal or is_truncated
        total_reward += reward

    if best_reward is None or total_reward > best_reward:
        best_reward = total_reward
        best_trajectory = env.trajectory

render(best_trajectory, [(target_x, target_y)], bounds)